In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ArxivRAG") \
    .getOrCreate()

# Load the JSON file
df = spark.read.json("reduce.json")

# Show schema
# df.printSchema()

# print(df.count())


# Show some sample rows
# df.select("id", "title", "abstract", "categories", "update_date", "versions").show(5, truncate=False)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 15:51:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:


# Load the JSON file
# df = spark.read.json("../arxiv-metadata-oai-snapshot.json")

# Show schema
# df.printSchema()

# print(df.count())
# df_copy = df.limit(10000)
# df_copy.write.format("json") \
#     .mode("append") \
#     .save("test3.json")

In [3]:
from pyspark.sql.functions import col, concat_ws, lower, regexp_replace, trim
from pyspark.sql.functions import monotonically_increasing_id, col


# Combine title and abstract into a 'document' field
df_cleaned = df.select(
    "id", "title", "abstract", "categories", "versions"
).withColumn(
    "document",
    concat_ws(" ", col("title"), col("abstract"))
).withColumn(
    "document",
    lower(regexp_replace(col("document"), r"[^a-zA-Z0-9\s]", ""))
).withColumn(
    "document", trim(col("document"))
).withColumn(
    "row_id", monotonically_increasing_id()
).withColumn(
    "year", col("versions")[0]["created"].substr(-17, 4)
)

# Filter out empty documents
df_cleaned = df_cleaned.filter(col("document") != "")

# Optional: Sample 10,000 rows for development
df_sample = df_cleaned.limit(5000)
# df_sample = df_cleaned

# delta = udf(lambda z: getDelta(z), IntegerType())
# Show a few processed rows
# df_sample.select("id", "categories", "document", "year", "versions").filter(df_sample.year > "1960" & df_sample.year < "2026").count()

In [4]:
# from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

import pandas as pd

# Convert Spark DataFrame to Pandas
pandas_df = df_sample.select("id", "document", "year").toPandas()

In [5]:
pandas_df.value_counts()
pandas_df[pandas_df["year"].apply(lambda x: int(x) > 1960 and int(x) < 2026)].value_counts()

id         document                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [6]:
# Load sentence transformer model
# model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipykernel_333094/2091687929.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/home/jeremy.dacruz2/.pyenv/versions/3.10.12/envs/arxiv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Embed the documents
# embeddings = model.encode(pandas_df['document'].tolist(), show_progress_bar=True)

metadata = [{"id": row["id"], "year": row["year"]} for _, row in pandas_df.iterrows()]
# print(metadata, "test")
vectorstore = Chroma.from_texts(pandas_df['document'].tolist(),embedding=embeddings,metadatas = metadata, ids=pandas_df['id'].tolist() ,persist_directory="../chroma_db")


[{'id': '0704.0001', 'year': '2007'}, {'id': '0704.0002', 'year': '2007'}, {'id': '0704.0003', 'year': '2007'}, {'id': '0704.0004', 'year': '2007'}, {'id': '0704.0005', 'year': '2007'}, {'id': '0704.0006', 'year': '2007'}, {'id': '0704.0007', 'year': '2007'}, {'id': '0704.0008', 'year': '2007'}, {'id': '0704.0009', 'year': '2007'}, {'id': '0704.0010', 'year': '2007'}, {'id': '0704.0011', 'year': '2007'}, {'id': '0704.0012', 'year': '2007'}, {'id': '0704.0013', 'year': '2007'}, {'id': '0704.0014', 'year': '2007'}, {'id': '0704.0015', 'year': '2007'}, {'id': '0704.0016', 'year': '2007'}, {'id': '0704.0017', 'year': '2007'}, {'id': '0704.0018', 'year': '2007'}, {'id': '0704.0019', 'year': '2007'}, {'id': '0704.0020', 'year': '2007'}, {'id': '0704.0021', 'year': '2007'}, {'id': '0704.0022', 'year': '2007'}, {'id': '0704.0023', 'year': '2007'}, {'id': '0704.0024', 'year': '2007'}, {'id': '0704.0025', 'year': '2007'}, {'id': '0704.0026', 'year': '2007'}, {'id': '0704.0027', 'year': '2007'}, 

In [ ]:

# df_sample = df_cleaned.filter(col("row_id") >= 100).drop("row_id").limit(10000)

# # Show a few processed rows
# df_sample.select("id", "categories", "document", "year").show(5, truncate=150)

In [ ]:
# Convert Spark DataFrame to Pandas
# pandas_df = df_sample.select("id", "document").toPandas()



# vectorstore.add_texts(pandas_df['document'].tolist())

In [ ]:
retriever = vectorstore.as_retriever()

# test query
query = "field"
retrieved_docs = retriever.get_relevant_documents(query)
i = 0
for doc in retrieved_docs:
    print(i, doc.page_content)
    print("\n")
    i +=1